# Working with boundary fluxes

In [2]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from NesrHydrusAnalyst import *

![title](../Assets/Domain.png)

## Reading the `6-BoundaryData.csv` files

In [3]:
extensions = ('', 'a', 'b', 'c', 'd', 'e')
sources = [f'../Datasets/H3D2_SandDitch0014{x}' for x in extensions]
dfs = [
    pd.read_csv(os.path.join(sources[x], 'Nesr', '6-BoundaryData.csv'))
    for x in range(len(extensions))
]
# Sample
dfs[0].head()

,Time_T,CumQAP_L3,CumQRP_L3,CumQA_L3,CumQR_L3,CumQ3_L3,CumQ1_L3,CumQS_L3,CumQ5_L3,CumQ6_L3,...,vKode7_L3pT,vKode8_L3pT,vKode9_L3pT,RunOff_LpT,Evapor_LpT,Infiltr_LpT,SnowLayer_L,dt,Iter,ItCum
0,0.0010,0.0,0.0,0.0,0.0,-0.0675,0.0,0.0,0.0,0.000188,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001000,2,2
1,0.0020,0.0,0.0,0.0,0.0,-0.1350,0.0,0.0,0.0,0.000371,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001000,2,4
2,0.0033,0.0,0.0,0.0,0.0,-0.2230,0.0,0.0,0.0,0.000601,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001300,2,6
3,0.0050,0.0,0.0,0.0,0.0,-0.3370,0.0,0.0,0.0,0.000889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001690,3,9
4,0.0072,0.0,0.0,0.0,0.0,-0.4850,0.0,0.0,0.0,0.001250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002197,3,12


### I found an error, that this is not the desired table for boundaries
The displayed table is for `Cum_Q.out, h_Mean.out, v_Mean.out, Run_inf.out` tables

While we need the outputs of `Boundary.out`

So, **Modify the `export_all_csvs` function**

In [22]:
def export_all_csvs(source_folder,
                    save_normal_grid=True,
                    pretty_output=True,
                    save_rotated_only=False,
                    rotation_angle=0,  #=2.2899
                    rotation_axis='y'):
    # Reading the files `MESHTRIA.TXT`, `TH.TXT`, `V.TXT`, `H.TXT`
    df = read_hydrus_data(
        folder=source_folder, save_to_csv=False, read_velocities=True)
    n = 0
    if not save_rotated_only:
        if save_normal_grid:
            n += 1
            save_this(df, source_folder, f'{n}-Original_Grid', authorized=True)
        if pretty_output:
            df_pr = reform_grid(df)
            n += 1
            save_this(
                df_pr, source_folder, f'{n}-Original_PrettyGrid', authorized=True)
    else:
        if rotation_angle != 0:
            df_rotated = rotate_back(
                df, rotation_angle, rotation_axis=rotation_axis)
            if save_normal_grid:
                n += 1
                save_this(
                    df_rotated,
                    source_folder,
                    f'{n}-Rotated_Grid',
                    authorized=True)
            if pretty_output:
                df_rotated = reform_grid(df_rotated)
                n += 1
                save_this(
                    df_rotated,
                    source_folder,
                    f'{n}-Rotated_PrettyGrid',
                    authorized=True)

    # If required, rotate the given dataframe

    # Combining the two files: `A_Level.out` and `ATMOSPH.IN`
    n += 1
    save_this(
        read_a_level_out(source_folder, geom='3D'),
        source_folder,
        f'{n}-Atm_A_level',
        authorized=True)

    # Saving info from one file: Balance.out
    n += 1
    save_this(
        read_balance_out(source_folder)[1],
        source_folder,
        f'{n}-MassBalance',
        authorized=True)

    # Saving eight files: SELECTOR.IN, DIMENSIO.IN, Run_Inf.out, Balance.out,
    # A_Level.out, ATMOSPH.IN, `Boundary.out` and `Check.out` files
    n += 1
    save_all_simulation_info(
        source_folder, saving_name=f'{n}-Simulation_info', save_type='csv')
    # Reading and combining Boundary.out tables
    n += 1
    save_this(
        read_boundary_data(source_folder, 19, 22, dt_frame=df),
        source_folder,
        f'{n}-BoundaryData',
        authorized=True)
    # Reading and combining Cum_Q.out, h_Mean.out, v_Mean.out, Run_inf.out tables
    n += 1
    save_this(
        get_mean_outs_table(source_folder),
        source_folder,
        f'{n}-CumulativeMeans',
        authorized=True)
    # Reading the file `Check.out` for materials properties
    n += 1
    save_this(
        get_materials_properties(source_folder),
        source_folder,
        f'{n}-MaterialsData',
        authorized=True)
    print('**All the CSV files were exported successfully**')
    pass

In [17]:
def read_boundary_data(folder='Current',
                       titles_loc=19,
                       data_begins=22,
                       nums_per_line=10,
                       save_to_csv=False,
                       dt_frame=None):
    '''
    A function to read both BOUNDARY.IN and  Boundary.outfiles from HYDRUS 
        outputs, then to:
            1- return one dataframe contains both data in a decent format.
            2- save this output to a CSV file (optional, True by default)
    Input:
        The name of the main folder (leave balank for the current folder)
        titles_loc=19, the line number of the first line contains captions
        data_begins=22, the line number of the first line contains data
        nums_per_line=10, teh number of columns per line in BOUNDARY.IN file
        The option to save_to csv, default =True (Boolean)
    '''

    # Specify the source folder
    if folder == 'Current':
        read_dir = os.getcwd()
    else:
        read_dir = folder
    if dt_frame is None:
        dt_frame= read_hydrus_data(folder=folder, 
                             save_to_csv=False, 
                             read_velocities=True)
    # Finding number of nodes in the file
    mesh_file = os.path.join(read_dir, 'BOUNDARY.IN')
    num_cells = int(linecache.getline(mesh_file, 4).split()[0])
    # Define dataframe titles
    titles = ['n', 'surface_area']

    def get_num_lines(num_cells, nums_per_line=10):
        num_lines = int(num_cells / nums_per_line)
        if num_cells % nums_per_line > 0: num_lines += 1
        return num_lines

    def read_snakey_list(file_name, start_line, end_line, data_type=float):
        '''
        Reading a list of numbers thar are stored in a text file, where
        there are `numbers_count` numbers, stored as `nums_per_line` numbers 
        per line, the starting position of first occurence of numbers is at
        `start_line` line number
        returns a numpy array of the data with the `data_type` given
        '''
        points = []
        for i in range(start_line, end_line):
            points.extend(linecache.getline(file_name, i).split())
        return np.array(points, data_type)

    num_lines = get_num_lines(num_cells, nums_per_line)

    first_line = 10
    end_line = first_line + num_lines
    points = read_snakey_list(mesh_file, first_line, end_line, data_type=int)

    first_line = end_line + 1
    end_line = first_line + num_lines
    areas = read_snakey_list(mesh_file, first_line, end_line)
    data = np.array([points, areas]).T
    data_in = pd.DataFrame(
        data, columns=titles).astype({
            "n": int,
            "surface_area": float
        })

    # Adding the surface area to the above table
    # print(num_cells, nums_per_line, num_lines)
    frames = []
    time_steps=get_available_timesteps(dt_frame)[1:]
    # print(time_steps)
    for t in time_steps:
        data_ends = data_begins + num_cells - 1
        # print(titles_loc, data_begins, num_lines, data_ends)
        dft = get_means_table(
            os.path.join(folder, 'Boundary.out'),
            titles_loc,
            data_begins,
            units_location=None,
            reading_end=data_ends,
            replace_units=False)  #.sample(3)
        # add a column for time
        dft = dft.assign(time_step=t)
        # convert all columns to numeric
        cols = list(dft)
        # print(cols)
        # display(dft)
        dft[cols] = dft[cols].apply(pd.to_numeric, errors='coerce')
        dft = pd.merge(dft, data_in, on='n')
        frames.append(dft)
        titles_loc = data_ends + 5
        data_begins = titles_loc + 3
    # display(frames)
    frames = pd.concat(frames, ignore_index=True)
    
    # Saving if allowed
    # save_this(dataframe, data_folder, output_name, authorized=True)
    if save_to_csv:
        save_this(frames, folder, 'boundary_data.CSV', authorized=save_to_csv)
    
    return frames  

In [15]:
read_boundary_data(sources[0], 19, 22)

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step,surface_area
0,1,9,0.0,0.00,25.0,-6,0.000,100000.0,-99.97,0.243,20.0,0.00,5,0.000000
1,2,12,0.0,30.00,25.0,-6,0.000,100000.0,-108.69,0.236,20.0,0.00,5,0.000000
2,3,13,0.0,0.00,28.0,-4,0.000,0.0,-22.31,0.098,20.0,0.00,5,1.125000
3,4,14,45.0,0.00,28.0,-3,0.750,-1.0,-3.09,0.413,20.0,3.75,5,0.749999
4,5,15,45.0,30.00,28.0,-3,0.375,-1.0,-3.21,0.410,20.0,1.87,5,0.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9211,764,4959,0.0,23.75,1.0,-6,0.000,100000.0,-13.16,0.165,20.0,0.00,2880,0.000000
9212,765,4960,0.0,25.00,1.0,-6,0.000,100000.0,-13.16,0.165,20.0,0.00,2880,0.000000
9213,766,4961,0.0,26.25,1.0,-6,0.000,100000.0,-13.16,0.165,20.0,0.00,2880,0.000000
9214,767,4962,0.0,27.50,1.0,-6,0.000,100000.0,-13.16,0.165,20.0,0.00,2880,0.000000


In [23]:
export_all_csvs(sources[0])

**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/1-Original_Grid.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/2-Original_PrettyGrid.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/3-Atm_A_level.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/4-MassBalance.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/5-Simulation_info.csv


PermissionError: [Errno 13] Permission denied: '../Datasets/H3D2_SandDitch0014\\Nesr\\6-BoundaryData.csv'